### Requirements

In [1]:
import pandas as pd
import requests
import json
from datetime import date
from dateutil import relativedelta
import random

# from newsapi-python import
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from bs4 import BeautifulSoup

### Scraping from API: https://newsapi.org/v2/

#### Getting a list of random dates within a range of 1 month from "today" and a list of 20 random sources out of 81 possible english language sources

In [2]:
def get_dates_list_one_month_range():
    lastmonth = date.today() + relativedelta.relativedelta(months=-1)
    delta_days = (date.today() - lastmonth).days
    dates_list = []

    for i in range(1,delta_days+1):
        dates_list.append((lastmonth + relativedelta.relativedelta(days=i)).strftime("%Y-%m-%d"))
    
    return dates_list

def random_sources():
    get_sources = open('../raw_data/news_api/sources.json',) 
    data = json.load(get_sources)
    get_sources.close()
    random_sources_list = []
    sources_df = pd.DataFrame(data["sources"])
    sources_df = sources_df[sources_df["language"] == "en"]
    sources_list = sources_df.id.to_list()
    # the maximum number of sources is 20 (there are 81 language sources in total)
    sources_list = random.sample(sources_list, 20)
    random_sources = ",".join(sources_list)
    
    return random_sources

#### Getting a request response from news API

In [20]:
api = "940f51da7bc0439b8cf24417ae0bd5e4" # API jmartins
nr_requests = 2

def get_random_everything_news_api(api, nr_requests):
    lastmonth = date.today() + relativedelta.relativedelta(months=-1)
    delta_days = (date.today() - lastmonth).days

    files_list = []
    date_iterator = get_dates_list_one_month_range()

    # the maximum number of requests per day is 100
    # the maximum number of news articles we can get per page is 100
    # Also, if we request 100 articles per page we can only retrieve one page max per request.

    for i in range(nr_requests):
        sources = random_sources()

        if i >= delta_days:
            date_iterator = random.sample(get_dates_list_one_month_range(),1)[0]
        else:
            date_iterator = get_dates_list_one_month_range()[i]
        
        url = 'https://newsapi.org/v2/'
        url_params = 'everything' #everything or top headlines
        params = { 'sources': sources,
                    'pageSize': 100,
                    'from': date_iterator,
                    'to': date_iterator,
                    'language':'en',
                    'apiKey': api}

        #import ipdb; ipdb.set_trace()
        response = requests.get(url + url_params, params=params)
        file = response.json()
        
        files_list.append(file["articles"])
            
        return files_list

get_random_everything_news_api(api, nr_requests)

0 2021-05-12
1 2021-05-13


[]

In [ ]:
        #import ipdb; ipdb.set_trace()
        response = requests.get(url + url_params, params=params)
        file = response.json()
        
        print("this_file")
        print()
        print(file)

        files_list.append(file["articles"])
        
        print("files_list")
        print()
        print(files_list)


### Putting api request result into a dataframe

In [6]:
files_list = get_random_everything_news_api(api, nr_requests)

In [7]:
elems =[]
for file in files_list:
    for elem in file:
        elems.append(elem)
news_api_df = pd.DataFrame(elems)
news_api_df = news_api_df.drop_duplicates(subset=['content'])
news_api_df['content'] = news_api_df['content'].replace(['\n','\r'],' ', regex=True)
news_api_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'the-verge', 'name': 'The Verge'}",Casey Newton,The child safety problem on platforms is worse...,A startling new report finds far more minors u...,https://www.theverge.com/2021/5/12/22432863/ch...,https://cdn.vox-cdn.com/thumbor/UKASaIesr3GBy_...,2021-05-13T00:00:00Z,Millions of young children are using platforms...
1,"{'id': 'the-times-of-india', 'name': 'The Time...",TNN,84% of vaccines sent abroad was part of licenc...,India News: NEW DELHI: Amid the ongoing politi...,https://timesofindia.indiatimes.com/india/84-o...,"https://static.toiimg.com/thumb/msid-47529300,...",2021-05-12T23:59:53Z,84% of vaccines sent abroad was part of licenc...
2,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,"Oygen situation grim, take steps to improve it...",India News: PANAJI: The Goa Bench of the Bomba...,https://timesofindia.indiatimes.com/india/oyge...,"https://static.toiimg.com/thumb/msid-82593549,...",2021-05-12T23:58:15Z,"Oygen situation grim, take steps to improve it..."
3,"{'id': 'the-times-of-india', 'name': 'The Time...",PTI,"O2 situation grim, take steps to improve it: H...",India News: PANAJI: The Goa Bench of the Bomba...,https://timesofindia.indiatimes.com/india/o2-s...,"https://static.toiimg.com/thumb/msid-82593549,...",2021-05-12T23:58:15Z,"O2 situation grim, take steps to improve it: H..."
4,"{'id': 'the-verge', 'name': 'The Verge'}",Jay Peters,Biden signs executive order aiming to prevent ...,President Joe Biden signed an executive order ...,https://www.theverge.com/2021/5/12/22433266/bi...,https://cdn.vox-cdn.com/thumbor/dgLoMcxivoI7hg...,2021-05-12T23:58:12Z,It arrives in the wake of multiple recent cybe...
...,...,...,...,...,...,...,...,...
95,"{'id': 'the-times-of-india', 'name': 'The Time...",Chaitanya Marpakwar,BMC floats global EoI to buy 50L vaccines,The BMC on Wednesday floated a global expressi...,https://timesofindia.indiatimes.com/city/mumba...,"https://static.toiimg.com/thumb/msid-82593486,...",2021-05-12T22:35:00Z,"BMC floats global EoI to buy 50L vaccines, 1st..."
96,"{'id': 'reuters', 'name': 'Reuters'}",Reuters,Delaying second COVID-19 vaccine doses can hel...,Giving a first dose of COVID-19 vaccine but de...,https://www.reuters.com/business/healthcare-ph...,https://www.reuters.com/resizer/oJoyREL2M11hIK...,2021-05-12T22:35:00Z,Giving a first dose of COVID-19 vaccine but de...
97,"{'id': 'abc-news', 'name': 'ABC News'}","Ben Gittleson, Justin Gomez, Luke Barr",Biden signs executive order to improve governm...,President Joe Biden signed an executive order ...,https://abcnews.go.com/Politics/biden-signs-ex...,https://s.abcnews.com/images/General/president...,2021-05-12T22:34:52Z,President Joe Biden signed an executive order ...
98,"{'id': 'politico', 'name': 'Politico'}",Eric Geller,Biden orders federal cyber upgrade after barra...,The far-reaching directive is an attempt to cl...,https://www.politico.com/news/2021/05/12/biden...,https://static.politico.com/a5/f2/651033d44f83...,2021-05-12T22:33:25Z,And it sets the stage for requiring federal co...


In [ ]:
## Alternative way to umpack file contents into concatenated "list contents" list.
# files_list = [file1, file2]
# files_list_unpacked = [*file1, *file2]
# news_api_df = pd.DataFrame([*file1, *file2])

### Info leftovers do scaping do José (em baixo)

In [ ]:
my_dict = {'CRIME': 'Crime',
 'ENTERTAINMENT': 'Entertainment',
 'WORLD NEWS': 'World News',
 'IMPACT': 'Other',
 'POLITICS': 'Politics',
 'WEIRD NEWS': 'Other',
 'BLACK VOICES': 'Activism',
 'WOMEN': 'Entertainment',
 'COMEDY': 'Entertainment',
 'QUEER VOICES': 'Activism',
 'SPORTS': 'Sports',
 'BUSINESS': 'Business',
 'TRAVEL': 'Culture',
 'MEDIA': 'Media',
 'TECH': 'Technology',
 'RELIGION': 'Religion',
 'SCIENCE': 'Science',
 'LATINO VOICES': 'Activism',
 'EDUCATION': 'Education',
 'COLLEGE': 'Education',
 'PARENTS': 'Other',
 'ARTS & CULTURE': 'Culture',
 'STYLE': 'Trends',
 'GREEN': 'Activism',
 'TASTE': 'Culture',
 'HEALTHY LIVING': 'Health',
 'THE WORLDPOST': 'World News',
 'GOOD NEWS': 'Other',
 'WORLDPOST': 'World News',
 'FIFTY': 'Other',
 'ARTS': 'Culture',
 'WELLNESS': 'Health',
 'PARENTING': 'Other',
 'HOME & LIVING': 'Trends',
 'STYLE & BEAUTY': 'Trends',
 'DIVORCE': 'Other',
 'WEDDINGS': 'Other',
 'FOOD & DRINK': 'Culture',
 'MONEY': 'Other',
 'ENVIRONMENT': 'Activism',
 'CULTURE & ARTS': 'Culture'}

In [ ]:
new_data = data.copy()
new_data['category'] = data.category.map(lambda x: my_dict[x])

In [ ]:
new_data.head()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(new_data.headline, show_progress_bar=True)
pd.DataFrame(embeddings).to_csv('../big_picture/data/embeddings_200k.csv')